<a href="https://colab.research.google.com/github/tyoc213/fastai_xla_extensions/blob/fix_prev_lenet/explore_nbs/PickableOpt_2_with_Basic_lenet_exploration_MultiTPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install fastai2 from github

In [ ]:
!pip install -U pandas --upgrade
!pip install -U fastcore --upgrade
!pip install -U fastai --upgrade 
#!pip install -Uqq git+https://github.com/tyoc213/fastai_xla_extensions@fix_prev_lenet

Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (1.1.2)
Requirement already up-to-date: fastcore in /usr/local/lib/python3.6/dist-packages (1.0.9)
Requirement already up-to-date: fastai in /usr/local/lib/python3.6/dist-packages (2.0.10)


In [ ]:
VERSION = "20200707"  #"20200515" @param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5115  100  5115    0     0  33000      0 --:--:-- --:--:-- --:--:-- 33000


In [ ]:
#!TORCH_SHOW_CPP_STACKTRACES=1 python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev
!python pytorch-xla-env-setup.py  --version $VERSION --apt-packages libomp5 libopenblas-dev

Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200707 ...
Uninstalling torch-1.7.0a0+12b5bdc:
  Successfully uninstalled torch-1.7.0a0+12b5bdc
Uninstalling torchvision-0.8.0a0+86b6c3e:
  Successfully uninstalled torchvision-0.8.0a0+86b6c3e
Copying gs://tpu-pytorch/wheels/torch-nightly+20200707-cp36-cp36m-linux_x86_64.whl...
\ [1 files][107.5 MiB/107.5 MiB]                                                
Operation completed over 1 objects/107.5 MiB.                                    
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200707-cp36-cp36m-linux_x86_64.whl...
\ [1 files][123.8 MiB/123.8 MiB]                                                
Operation completed over 1 objects/123.8 MiB.                                    
Copying gs://tpu-pytorch/wheels/torchvision-nightly+20200707-cp36-cp36m-linux_x86_64.whl...
/ [1 files][  2.2 MiB/  2.2 MiB]                                                
Operation completed over 1 objects/2.2 MiB.        

In [ ]:
!pip freeze | grep torch 
!pip freeze | grep fast

torch==1.7.0a0+12b5bdc
torch-xla==1.6+5430aca
torchsummary==1.5.1
torchtext==0.3.1
torchvision==0.8.0a0+86b6c3e
fastai==2.0.10
fastcore==1.0.9
fastdtw==0.3.4
fastprogress==1.0.0
fastrlock==0.5


In [ ]:
#import fastai_xla_extensions.core

In [ ]:
from fastai.vision.all import *

In [ ]:
default_device()

device(type='cpu')

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path; path.ls()

(#3) [Path('labels.csv'),Path('valid'),Path('train')]

In [ ]:
(path/'train').ls()

(#2) [Path('train/3'),Path('train/7')]

# multi TPU

In [13]:
# Configures training (and evaluation) parameters
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import torch_xla.distributed.parallel_loader as pl
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
from fastai.vision.all import *
import time
from fastai.test_utils import *
print(f'torch version {torch.__version__}')

import pdb

path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path; path.ls()


def debug_on(*exceptions):
    if not exceptions:
        exceptions = (AssertionError, )
    def decorator(f):
        @functools.wraps(f)
        def wrapper(*args, **kwargs):
            try:
                return f(*args, **kwargs)
            except exceptions:
                pdb.post_mortem(sys.exc_info()[2])
        return wrapper
    return decorator

class Lenet2(nn.Module):
    def __init__(self):
        super(Lenet2, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2) # Only 2 outputs instead of 10
    @debug_on(KeyError)
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    @debug_on(KeyError)
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features









# third level patch... alll!!!!

class _BaseOptimizer():
    "Common functionality between `Optimizer` and `OptimWrapper`"
    @debug_on(KeyError)
    def __getattr__(self, name):
      #print(f"================= = = = = = ORIGINAL BASE OPTIMIZER {name}  = = = = = =================")
      return getattr(self, name)
    @debug_on(KeyError)
    def all_params(self, n=slice(None), with_grad=False):
        #print(f"================= = = = = = ORIGINAL BASE OPTIMIZER def all_params(self, n=slice(None), with_grad=False):  = = = = = =================")
        #print(f"================= = = = = = {type(self)} {dir(self)}  = = = = = =================")
        #print(f"================= = = = = = {type(self.param_groups)} {dir(self.param_groups)}  = = = = = =================")
        ###print(f"================= = = = = = {self.param_groups}  = = = = = =================")
        # TODO: unroll this loop
        res = L((p,pg,self.state[p],hyper) for pg,hyper in zip(self.param_lists[n],self.hypers[n]) for p in pg)        
        
        return L(o for o in res if o[0].grad is not None) if with_grad else res

    @debug_on(KeyError)
    def _set_require_grad(self, rg, p,pg,state,h): p.requires_grad_(rg or state.get('force_train', False))
    @debug_on(KeyError)
    def freeze_to(self, n):
        self.frozen_idx = n if n >= 0 else len(self.param_lists) + n
        if self.frozen_idx >= len(self.param_lists):
            warn(f"Freezing {self.frozen_idx} groups; model has {len(self.param_lists)}; whole model is frozen.")
        for o in self.all_params(slice(n, None)): self._set_require_grad(True,  *o)
        for o in self.all_params(slice(None, n)): self._set_require_grad(False, *o)

    @debug_on(KeyError)
    def freeze(self):
        assert(len(self.param_lists)>1)
        self.freeze_to(-1)

    @debug_on(KeyError)
    def set_freeze(self, n, rg, ignore_force_train=False):
        for p in self.param_lists[n]: p.requires_grad_(rg or (state.get('force_train', False) and not ignore_force_train))

    @debug_on(KeyError)
    def unfreeze(self): self.freeze_to(0)
    @debug_on(KeyError)
    def set_hypers(self, **kwargs): L(kwargs.items()).starmap(self.set_hyper)
    @debug_on(KeyError)
    def _set_hyper(self, k, v):
        for v_,h in zip(v, self.hypers): h[k] = v_

    @debug_on(KeyError)
    def set_hyper(self, k, v):
        if isinstance(v, slice):
            if v.start: v = even_mults(v.start, v.stop, len(self.param_lists))
            else: v = [v.stop/10]*(len(self.param_lists)-1) + [v.stop]
        v = L(v, use_list=None)
        if len(v)==1: v = v*len(self.param_lists)
        assert len(v) == len(self.hypers), f"Trying to set {len(v)} values for {k} but there are {len(self.param_lists)} parameter groups."
        self._set_hyper(k, v)

    @property
    def param_groups(self):
      #print('%%%%%%%%%%%%%%%%%%%% P A R A M     G R O U P S %%%%%%%%%%%%%%%%%%%%')
      return [{**{'params': pg}, **hp} for pg,hp in zip(self.param_lists, self.hypers)]
    
    @param_groups.setter
    def param_groups(self, v):
        #print('%%%%%%%%%%%%%%%%%%%% P A R A M     G R O U P S      S E T T E R %%%%%%%%%%%%%%%%%%%%')
        for pg,v_ in zip(self.param_lists,v): pg = v_['params']
        for hyper,v_ in zip(self.hypers,v):
            for k,t in v_.items():
                if k != 'params': hyper[k] = t
    @debug_on(KeyError)
    def __setstate__(self, d):
      #print('%%%%%%%%%%%%%%%%%%%%****** ####### set state baseeeeee')
      self.__dict__.update(d) # 
      #print('%%%%%%%%%%%%%%%%%%%%****** ####### set state ENDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD')
    @debug_on(KeyError)
    def __getstate__(self):
      #print('%%%%%%%%%%%%%%%%%%%%****** ####### get state baseeeeee')
      d = dict(self.__dict__)
      return d
# Cell
@debug_on(KeyError)
def _update(state, new=None):
    if new is None: return state
    if isinstance(new, dict): state.update(new)
    return state

# Cell
@log_args(but='params,cbs,defaults')
class Optimizer(_BaseOptimizer):
    "Base optimizer class for the fastai library, updating `params` with `cbs`"
    _keep_on_clear = ['force_train', 'do_wd']
    @debug_on(KeyError)
    def __getattr__(self, name):
      #print(f"================= = = = = = BASE OPTIMIZER {name}  = = = = = =================")
      return getattr(self, name)
    def __init__(self, params, cbs, train_bn=True, **defaults):
        #print(f"================= = = = = = OPTIMIZER(_BaseOptimizer)  = = = = = =================\n"*10)
        params = L(params)
        self.cbs,self.state,self.train_bn = L(cbs),defaultdict(dict),train_bn
        defaults = merge(*self.cbs.attrgot('defaults'), defaults)
        self.param_lists = L(L(p) for p in params) if isinstance(params[0], (L,list)) else L([params])
        self.hypers = L({} for _ in range_of(self.param_lists))
        self.set_hypers(**defaults)
        self.frozen_idx = 0

    @debug_on(KeyError)
    def zero_grad(self):
        for p,*_ in self.all_params(with_grad=True):
            p.grad.detach_()
            p.grad.zero_()

    @debug_on(KeyError)
    def step(self):
        #print(f"================= = = = = = BASE OPTIMIZER STEPPPPPPPPPPP P P P  P PP P P P P  = = = = = =================")
        for p,pg,state,hyper in self.all_params(with_grad=True):
            for cb in self.cbs: state = _update(state, cb(p, **{**state, **hyper}))
            self.state[p] = state

    @debug_on(KeyError)
    def clear_state(self):
        for p,pg,state,hyper in self.all_params():
            self.state[p] = {k: state[k] for k in self._keep_on_clear if k in state}

    @debug_on(KeyError)
    def state_dict(self):
        state = [self.state[p] for p,*_ in self.all_params()]
        return {'state': state, 'hypers': self.hypers}

    @debug_on(KeyError)
    def load_state_dict(self, sd):
        assert len(sd["hypers"]) == len(self.param_lists)
        assert len(sd["state"])  == sum([len(pg) for pg in self.param_lists])
        self.hypers = sd['hypers']
        self.state = {p: s for p,s in zip(self.all_params().itemgot(0), sd['state'])}


### second patch
#@patch
#def param_groups(self:Optimizer): return [{**{'params': pg}, **hp} for pg,hp in zip(self.param_lists, self.hypers)]
#@patch
#def param_groups(self:Optimizer, v):
#        for pg,v_ in zip(self.param_lists,v): pg = v_['params']
#        for hyper,v_ in zip(self.hypers,v):
#            for k,t in v_.items():
#                if k != 'params': hyper[k] = t



# created to copy optimizer and have `__get_state__` and `__set_state__`
@debug_on(KeyError)
@patch
def step(self:Optimizer):
  #print(f'=========== * = ⁰ = * = * = step optimizer 0')
  for p,pg,state,hyper in self.all_params(with_grad=True):
    #print(f'=========== * = ⁰ = * = * = step optimizer 1')
    for cb in self.cbs:
      #print(f'=========== * = ⁰ = * = * = step optimizer INTERNAL')
      state = _update(state, cb(p, **{**state, **hyper}))
    #print(f'=========== * = ⁰ = * = * = step optimizer UPDATE')
    self.state[p] = state
    #print(f'=========== * = ⁰ = * = * = step optimizer ENDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD ENDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD ENDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD ENDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD ENDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD ENDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD ENDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD ENDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD ENDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD ENDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD ENDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD')
























# the new thing!!!! a PiclableOptimizer that has __getstate__() and __setstate__() this last is not needed AFAIK

class PickableOpt(Optimizer):
  def __init__(self, opt):
    #print('############## %%%%%%%%%%%%%%%%%%%%%%%%%%% deep copy from optimizer!!!!')
    self.__dict__ = deepcopy(opt.__dict__)
    self.opt = opt

  def __getstate__(self):
    #print('############## %%%%%%%%%%%%%%%%%%%%%%%%%%% PickableOpt#__getstate__!!! get state xla opt callback')
    v = vars(self)
#    v['param_groups'] = lambda x: return torch.zeros(7)
    #print(f'vars type are: {type(v)}')
    #print(f'vars type are: {len(v)}')
    #print(f'vars type are: {v.keys()}')
    #print(f'vars are: {v}')
    #print(f'vars are: {v}')
    v = self.state_dict()
    v['param_groups'] = self.param_groups
    return v




















































#### first patch

class XLAOptimProxy2:
    "Proxy optimizer to override `opt.step` with Pytorch XLA sync method `xm.optimizer_step` "  
    def __init__(self,opt, barrier=True):
        #print(f"================= = = = = = XLAOptimProxy2 = = = = = =================")
        self.opt = PickableOpt(opt)
        self._barrier = barrier


    @debug_on(KeyError)
    def xla_step(self):
        #print('step....')
        xm.optimizer_step(self.opt,barrier=self._barrier) # sync on gradient update

    @debug_on(KeyError)        
    def __getattr__(self,name):
        #print('*** *** ***',name)
#        print(f'***** ***** {name} {getattr(self.op, name, "*** no lo tiene")}')
        if name == 'step': # override proxying for step
            return getattr(self,'xla_step')
        if name in ('barrier','_barrier'):
            return getattr(self,name)
      
        # proxy everything else
        return getattr(self.opt,name)

    @property
    def barrier(self): return self._barrier

    @barrier.setter
    def barrier(self,v): self._barrier = v
  
class XLAOptCallback2(Callback):
    'Callback to replace `opt.step` with `xm.optimizer_step(opt)` as required to run on TPU'
    def __init__(self, barrier=True):
        self._barrier = barrier

    @debug_on(KeyError)
    def before_fit(self):
        'replace opt with proxy which calls `xm.optimizer_step` instead of `opt.step` and set `dls.device` and model to `xla_device`'
        to_device(self.dls, device=xm.xla_device())
        self.model.to(self.dls.device)
        if self.learn.opt is not None:
            if not isinstance(self.learn.opt,XLAOptimProxy2):
                opt = self.learn.opt
                self.learn.opt = XLAOptimProxy2(opt, barrier=self._barrier)

    @debug_on(KeyError)
    def after_fit(self):
        'restore original opt '
        if isinstance(self.learn.opt, XLAOptimProxy2):
            opt = self.learn.opt.opt
            self.learn.opt = opt







def map_fn(index, flags):
  # from fastai.callback.all import *
  dede = xm.xla_device()
  print(f'index is {index} and flags are {flags}')
  #xm.rendezvous('init')

  if not xm.is_master_ordinal():
    print(f"this is {dede}:{index} entering download once")
    xm.rendezvous('download_only_once')
    
  dblock = DataBlock(
    splitter = GrandparentSplitter(),
    item_tfms = Resize(28),
    blocks = (ImageBlock, CategoryBlock),
    get_items = get_image_files,
    get_y = parent_label,
    batch_tfms = []
  )
  if xm.is_master_ordinal():
    xm.master_print(f'this is {dede} exiting download once')
    xm.rendezvous('download_only_once')
  xm.master_print('creating lenet_tpu')
  lenet_tpu = Lenet2()
  xm.master_print('lenet created, goiing for dls_tpu')
  dls_tpu = dblock.dataloaders(path, device=dede)
  xm.master_print(f'creating learner!!! for {dede}')
  

  tpu_learner = Learner(dls_tpu,
                      lenet_tpu,
                      metrics=accuracy, 
                      loss_func=F.cross_entropy,
                      cbs=[])
  print(f"################ fit for {dede}")
  xm.master_print(f'***** fit for {dede}')
  tpu_learner.fit(1, cbs=[XLAOptCallback2()])
  xm.master_print(f'***** end fit for {dede}')
  t = torch.randn((2, 2), device=dede)
  print("################Process", index ,"is using", xm.xla_real_devices([str(dede)])[0])






# https://stackoverflow.com/a/9929970/682603
# excepthook
# 
import traceback
import logging
import os, sys

def my_excepthook(excType, excValue, traceback, logger):
    print("=== *** @@@ ### %%% === *** @@@ ### %%% === *** @@@ ### %%% === *** @@@ ### %%% === *** @@@ ### %%% === *** @@@ ### %%% Logging an uncaught exception",
                 exc_info=(excType, excValue, traceback))

sys.excepthook = my_excepthook
sys.unraisablehook = my_excepthook
##############threading.excepthook
#https://docs.python.org/3/library/sys.html#sys.excepthook




print('launching n procs')

flags={}
flags['batch_size'] = 32
flags['num_workers'] = 8
flags['num_epochs'] = 1
flags['seed'] = 1234

xmp.spawn(map_fn, args=(flags,), nprocs=8, start_method='fork')
print('end of launch')

torch version 1.7.0a0+12b5bdc
launching n procs
index is 0 and flags are {'batch_size': 32, 'num_workers': 8, 'num_epochs': 1, 'seed': 1234}
this is xla:1 exiting download once
index is 5 and flags are {'batch_size': 32, 'num_workers': 8, 'num_epochs': 1, 'seed': 1234}
this is xla:0:5 entering download once
index is 7 and flags are {'batch_size': 32, 'num_workers': 8, 'num_epochs': 1, 'seed': 1234}
this is xla:0:7 entering download once
index is 6 and flags are {'batch_size': 32, 'num_workers': 8, 'num_epochs': 1, 'seed': 1234}
this is xla:0:6 entering download once
index is 1 and flags are {'batch_size': 32, 'num_workers': 8, 'num_epochs': 1, 'seed': 1234}
this is xla:0:1 entering download once
index is 3 and flags are {'batch_size': 32, 'num_workers': 8, 'num_epochs': 1, 'seed': 1234}
this is xla:0:3 entering download once
index is 2 and flags are {'batch_size': 32, 'num_workers': 8, 'num_epochs': 1, 'seed': 1234}
this is xla:0:2 entering download once
index is 4 and flags are {'batc

epoch,train_loss,valid_loss,accuracy,time
0,0.699662,0.699772,0.495584,03:13


epoch,train_loss,valid_loss,accuracy,time
0,0.699113,0.699772,0.495584,03:13


################ fit for xla:0


epoch,train_loss,valid_loss,accuracy,time
0,0.699263,0.699772,0.495584,03:14


...

################ fit for xla:0


epoch,train_loss,valid_loss,accuracy,time
0,0.699018,0.699772,0.495584,03:13


################ fit for xla:0


epoch,train_loss,valid_loss,accuracy,time
0,0.701533,0.699772,0.495584,03:12


################Process 1 is using TPU:1
***** end fit for xla:1
################Process 0 is using TPU:0
################Process 2 is using TPU:2
################Process 4 is using TPU:4
################Process 7 is using TPU:7
################Process 3 is using TPU:3
################Process 6 is using TPU:6
################Process 5 is using TPU:5
end of launch
